# [텐서](https://tutorials.pytorch.kr/beginner/basics/tensorqs_tutorial.html)
- https://subinium.github.io/pytorch-Tensor-Variable/

In [1]:
import torch
import numpy as np

### 텐서 생성

In [3]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
x_data

tensor([[1, 2],
        [3, 4]])

In [5]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
np_array, x_np

(array([[1, 2],
        [3, 4]]),
 tensor([[1, 2],
         [3, 4]], dtype=torch.int32))

In [6]:
x_ones = torch.ones_like(x_data) # x_data의 속성을 유지합니다.
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # x_data의 속성을 덮어씁니다.
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.3421, 0.2420],
        [0.3782, 0.5077]]) 



In [7]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.9897, 0.7628, 0.2327],
        [0.7702, 0.7828, 0.8800]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [9]:
tensor = torch.rand(3,4)

print(f"Random Tensor: \n {tensor} \n")
print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Random Tensor: 
 tensor([[0.1392, 0.2380, 0.1032, 0.6953],
        [0.7158, 0.3913, 0.0911, 0.1545],
        [0.2038, 0.0664, 0.9982, 0.2105]]) 

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


### cuda 사용

In [10]:
# GPU가 존재하면 텐서를 이동합니다
if torch.cuda.is_available():
  tensor = tensor.to('cuda')

### tensor 차원 재구성
- reshape(), view()
> 둘 모두 문법은 동일합니다. x.view(1,2,3), x.reshape(1,2,3)처럼 변경하고 싶은 shape을 tuple로 입력으로 주면 됩니다.  
> 여기서는 정말 미세한 차이가 있습니다. 
> - `view`는 기존의 데이터와 같은 메모리 공간을 공유하며 stride 크기만 변경하여 보여주기만 다르게 합니다. 그래서 contiguous 해야만 동작하며, 아닌 경우 에러가 발생합니다.  
> - `reshape`는 메머리가 기존 Tensor와 동일한 메모리를 공유하는게 보장되지만 reshape은 그렇지 않습니다. 
> - 안전하게 형태만 바꾸고 싶다 -> reshape
> - 메모리가 공유되어 업데이트에 대한 보장이 이뤄진다 -> view  
  
- transpose(), permute()
> 종종 channel 차원을 마지막으로 보내야하는 순간이 존재하고, 연산에 따라 차원 간의 순서를 변경해줄 필요가 있습니다.  
> - `transpose()`: 2개의 차원을 변경하는데 사용
> - `permute()`: 모든 차원의 순서를 재배치

In [20]:
x =torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
print(f'x: {x}')

# view, reshape
v = x.view(3,2,2)
print(f'v: {v}')
r = v.reshape(-1)
print(f'r: {r}')

# transpose, permute
t = v.permute(1,2,0)
print(f't: {t}')

x: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
v: tensor([[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11]]])
r: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
t: tensor([[[ 0,  4,  8],
         [ 1,  5,  9]],

        [[ 2,  6, 10],
         [ 3,  7, 11]]])


In [25]:
t = np.zeros((4, 4, 3))
ft = torch.FloatTensor(t)
print(f'ft: {ft.shape}')
v1 = ft.view([-1, 3]) # ft라는 텐서를(?, 3)의 크기로 변경
print(f'v1: {v1.shape}')
v2 = ft.view([-1, 2, 3]) # ft라는 텐서를 (?, 2, 3)의 크기로 변경
print(f'v2: {v2.shape}')

ft: torch.Size([4, 4, 3])
v1: torch.Size([16, 3])
v2: torch.Size([8, 2, 3])


### 더미 차원 추가와 삭제: squeeze(), unsqueeze()
`Tensor`는 결국 모델에 넣기 위한 data의 변형이며 data에 따라 Tensor의 차원도 달라집니다. 보통 모델에 넣는 데이터의 shape은 다음과 같습니다.
- signal data `N, C, L`: batch, channel, length
- image data `N, C, H, W`: batch, channel, height, weight  
  
하지만 inference시 batchsize가 1이 되어 shape이 안맞는 경우, 아니면 후에 Conv연산을 위해 가상 channel이 필요한 경우, tensor간 연산을 위해 channel을 추가해야하는 경우 등등 size가 1인 차원을 추가하거나 없애야하는 상황이 많습니다.  
- squeeze(): 차원의 size가 1인 차원을 없애줌
  - 여러개의 차원 중 특정 차원만 지우고 싶다면, x.squeesz(1)처럼 사용, 아니면 모두 제거
- unsqueeze(): 차원 size가 1인 차원을 생성
  - idx번째 차원을 만들면 기존 idx차원부터 한칸씩 미뤄짐

### tensor 연산

In [13]:
tensor = torch.rand(4, 4)

print(f"Random Tensor: \n {tensor} \n")
print('First row: ',tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[:, -1])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

Random Tensor: 
 tensor([[0.2538, 0.8653, 0.8758, 0.4422],
        [0.1578, 0.2791, 0.3726, 0.8337],
        [0.8387, 0.5881, 0.9932, 0.2651],
        [0.8919, 0.9574, 0.8619, 0.8216]]) 

First row:  tensor([0.2538, 0.8653, 0.8758, 0.4422])
First column:  tensor([0.2538, 0.1578, 0.8387, 0.8919])
Last column: tensor([0.4422, 0.8337, 0.2651, 0.8216])
Last column: tensor([0.4422, 0.8337, 0.2651, 0.8216])
tensor([[0.2538, 0.0000, 0.8758, 0.4422],
        [0.1578, 0.0000, 0.3726, 0.8337],
        [0.8387, 0.0000, 0.9932, 0.2651],
        [0.8919, 0.0000, 0.8619, 0.8216]])


In [14]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[0.2538, 0.0000, 0.8758, 0.4422, 0.2538, 0.0000, 0.8758, 0.4422, 0.2538,
         0.0000, 0.8758, 0.4422],
        [0.1578, 0.0000, 0.3726, 0.8337, 0.1578, 0.0000, 0.3726, 0.8337, 0.1578,
         0.0000, 0.3726, 0.8337],
        [0.8387, 0.0000, 0.9932, 0.2651, 0.8387, 0.0000, 0.9932, 0.2651, 0.8387,
         0.0000, 0.9932, 0.2651],
        [0.8919, 0.0000, 0.8619, 0.8216, 0.8919, 0.0000, 0.8619, 0.8216, 0.8919,
         0.0000, 0.8619, 0.8216]])


In [15]:
t2 = torch.cat([tensor, tensor, tensor], dim=0)
print(t2)

tensor([[0.2538, 0.0000, 0.8758, 0.4422],
        [0.1578, 0.0000, 0.3726, 0.8337],
        [0.8387, 0.0000, 0.9932, 0.2651],
        [0.8919, 0.0000, 0.8619, 0.8216],
        [0.2538, 0.0000, 0.8758, 0.4422],
        [0.1578, 0.0000, 0.3726, 0.8337],
        [0.8387, 0.0000, 0.9932, 0.2651],
        [0.8919, 0.0000, 0.8619, 0.8216],
        [0.2538, 0.0000, 0.8758, 0.4422],
        [0.1578, 0.0000, 0.3726, 0.8337],
        [0.8387, 0.0000, 0.9932, 0.2651],
        [0.8919, 0.0000, 0.8619, 0.8216]])


In [19]:
t2[:,0]

tensor([0.2538, 0.1578, 0.8387, 0.8919, 0.2538, 0.1578, 0.8387, 0.8919, 0.2538,
        0.1578, 0.8387, 0.8919])

In [20]:
# 두 텐서 간의 행렬 곱(matrix multiplication)을 계산합니다. y1, y2, y3은 모두 같은 값을 갖습니다.
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(tensor)
torch.matmul(tensor, tensor.T, out=y3)

tensor([[1.0270, 0.7351, 1.2000, 1.3445],
        [0.7351, 0.8588, 0.7235, 1.1468],
        [1.2000, 0.7235, 1.7601, 1.8218],
        [1.3445, 1.1468, 1.8218, 2.2132]])

In [24]:
# 요소별 곱(element-wise product)을 계산합니다. z1, z2, z3는 모두 같은 값을 갖습니다.
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[0.0644, 0.0000, 0.7670, 0.1956],
        [0.0249, 0.0000, 0.1389, 0.6951],
        [0.7033, 0.0000, 0.9864, 0.0703],
        [0.7954, 0.0000, 0.7428, 0.6749]])

In [25]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

7.608250617980957 <class 'float'>


In [26]:
print(tensor, "\n")
tensor.add_(5)
print(tensor)

tensor([[0.2538, 0.0000, 0.8758, 0.4422],
        [0.1578, 0.0000, 0.3726, 0.8337],
        [0.8387, 0.0000, 0.9932, 0.2651],
        [0.8919, 0.0000, 0.8619, 0.8216]]) 

tensor([[5.2538, 5.0000, 5.8758, 5.4422],
        [5.1578, 5.0000, 5.3726, 5.8337],
        [5.8387, 5.0000, 5.9932, 5.2651],
        [5.8919, 5.0000, 5.8619, 5.8216]])


In [27]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [28]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


In [29]:
n = np.ones(5)
t = torch.from_numpy(n)

In [30]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
